In [1]:
import yaml
import numpy as np
from math import sqrt

import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_html
%matplotlib inline

INFO:commons:version: 6.1.5
INFO:typing:version: 6.1.8
DEBUG:duckietown_world:duckietown-world version 6.2.7 path /home/chrigi/Master-Thesis-Code/driving-games/venv/lib/python3.8/site-packages
INFO:geometry:version: 2.0.5


# Double Lanes

### Definition of lanes that stay parallel

In [2]:
# load a empty tile to display the lanes
map_data_yaml = """
tile_size: 1

tiles:
  - [asphalt]

"""
# load the map containing one tile using the duckietown world module
map_data = yaml.load(map_data_yaml, Loader=yaml.SafeLoader)
m = dw.construct_map(map_data)

# extract the tile size
tile_size=map_data["tile_size"]

width_old = 0.376  # width of lanes in duckietown world
width = width_old / 2  # creating a tile using double lanes, take only half the lane size

# the hard coded values are all taken from the duckietown world module.
# I tried to scale the space between two lanes accordingly to the new lane width
width_between_lanes_old = 0.22 * 2 - width_old
width_between_lanes = width_between_lanes_old / 2

"""
Get the coordinates of the control points at the edge of a tile
                   ____________________ (0.5, 0.5)
               y4  |                  |
               y3  |        y         |
                   |        ^         |
                   |        |--> x    |
               y2  |                  |
               y1  |                  |
     (-0.5, -0.5)  --------------------
                     x1 x2      x3 x4
"""
y2 = -(width_between_lanes / 2 + width / 2)
y1 = y2 - (width_between_lanes + width)

y3, y4 = -y2, -y1

x1, x2, x3, x4 = y1, y2, y3, y4

half_tile = tile_size / 2

# double straight lanes
ctr_pts1 = [  # lane on the right
    dw.SE2Transform([-half_tile, y1], np.deg2rad(0)),
    dw.SE2Transform([half_tile, y1], np.deg2rad(0)),
]
ctr_pts2 = [  # lane on the left
    dw.SE2Transform([-half_tile, y2], np.deg2rad(0)),
    dw.SE2Transform([half_tile, y2], np.deg2rad(0)),

]

def midpoints_curve(start, end):
    """
    This functions calculates the coordinates of the control point in the the middle of a curve,
    which is perfectly circular
    """
    x1, y1 = start
    x2, y2 = end
    
    r = x2 - x1
    dr = (1-sqrt(2)/2) * r
    if y2 > y1:
        res = [x2 - dr, y1 + dr]
    else:
        res = [x2 - dr, y1 - dr]

    return [round(_, 3) for _ in res]
    
    
#double left lane
start = [-half_tile, y1]
end = [x4, half_tile]
middle = midpoints_curve(start, end)
ctr_pts3 = [  # lane on the right
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(middle, np.deg2rad(45)),
    dw.SE2Transform(end, np.deg2rad(90)),
]

start = [-half_tile, y2]
end = [x3, half_tile]
middle = midpoints_curve(start, end)
ctr_pts4 = [  # lane on the left
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(middle, np.deg2rad(45)),
    dw.SE2Transform(end, np.deg2rad(90)),
]

# double right lane
start = [-half_tile, y1]
end = [x1, -half_tile]
middle = midpoints_curve(start, end)
ctr_pts5 = [  # lane on the right
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(middle, np.deg2rad(-45)),
    dw.SE2Transform(end, np.deg2rad(-90)),
]

start = [-half_tile, y2]
end = [x2, -half_tile]
middle = midpoints_curve(start, end)
ctr_pts6 = [ # lane on the left
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(middle, np.deg2rad(-45)),
    dw.SE2Transform(end, np.deg2rad(-90)),
]

# get the lane as a lane segment
test_lane1 = dw.LaneSegment(width, ctr_pts1)
test_lane2 = dw.LaneSegment(width, ctr_pts2)
test_lane3 = dw.LaneSegment(width, ctr_pts3)
test_lane4 = dw.LaneSegment(width, ctr_pts4)
test_lane5 = dw.LaneSegment(width, ctr_pts5)
test_lane6 = dw.LaneSegment(width, ctr_pts6)

# On a duckietown map the origin of the coordinate system is on the bottom left.
# In order to display the lanes which have been defined in the their original coordinate system
# (tile coordinate system) one has to move the lanes when their are placed on the empty one-tile map
gt = dw.SE2Transform([half_tile, half_tile], 0)
m.set_object("Test Lane 1", test_lane1, ground_truth=gt)
m.set_object("Test Lane 2", test_lane2, ground_truth=gt)
m.set_object("Test Lane 3", test_lane3, ground_truth=gt)
m.set_object("Test Lane 4", test_lane4, ground_truth=gt)
m.set_object("Test Lane 5", test_lane5, ground_truth=gt)
m.set_object("Test Lane 6", test_lane6, ground_truth=gt)
ipython_draw_html(m);

INFO:duckietown_world: data: /home/chrigi/Master-Thesis-Code/driving-games/venv/lib/python3.8/site-packages/duckietown_world/data
INFO:duckietown_world: area: RectangularArea(pmin=[0. 0.],pmax=[1. 1.])
DEBUG:commons.fs:Written 0.1MB to: out/ipython_draw_html/140391933878080/drawing.html
INFO:duckietown_world:Written SVG
 fn_svg: out/ipython_draw_html/140391933878080/drawing.svg


### Define the lanes used for lane changes

In [3]:
# double straight lane change
ctr_pts7 = [  # from right lane to left lane
    dw.SE2Transform([-half_tile, y1], np.deg2rad(0)),
    dw.SE2Transform([-half_tile /2 , y1], np.deg2rad(0)),
    dw.SE2Transform([0, (y1 + y2) / 2], np.deg2rad(45)),
    dw.SE2Transform([half_tile / 2, y2], np.deg2rad(0)),
    dw.SE2Transform([half_tile, y2], np.deg2rad(0)),
]
ctr_pts8 = [  # left to right
    dw.SE2Transform([-half_tile, y2], np.deg2rad(0)),
    dw.SE2Transform([-half_tile / 2, y2], np.deg2rad(0)),
    dw.SE2Transform([0, (y1 + y2) / 2], np.deg2rad(-45)),
    dw.SE2Transform([half_tile / 2, y1], np.deg2rad(0)),
    dw.SE2Transform([half_tile, y1], np.deg2rad(0)),

]

#double left lane change
start = [-half_tile, y1]
start2 = [x3 - width / 2, y1]
end2 = [x3, y1 + width / 2]
middle2 = midpoints_curve(start2, end2)
end = [x3, half_tile]

ctr_pts9 = [  # right to left
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(start2, np.deg2rad(0)),
    dw.SE2Transform(middle2, np.deg2rad(45)),
    dw.SE2Transform(end2, np.deg2rad(90)),
    dw.SE2Transform(end, np.deg2rad(90)),
]

start = [-half_tile, y2]
start2 = [x4 - width / 2, y2]
end2 = [x4, y2 + width / 2]
middle2 = midpoints_curve(start2, end2)
end = [x4, half_tile]
middle = midpoints_curve(start, end)
ctr_pts10 = [  # left to right
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(start2, np.deg2rad(0)),
    dw.SE2Transform(middle2, np.deg2rad(45)),
    dw.SE2Transform(end2, np.deg2rad(90)),
    dw.SE2Transform(end, np.deg2rad(90)),
]

# double right lane change
start = [-half_tile, y1]
start2 = [x2 - width / 2, y1]
end2 = [x2, y1 - width / 2]
middle2 = midpoints_curve(start2, end2)
end = [x2, -half_tile]
middle = midpoints_curve(start, end)
ctr_pts11 = [  # right to left
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(start2, np.deg2rad(0)),
    dw.SE2Transform(middle2, np.deg2rad(-45)),
    dw.SE2Transform(end2, np.deg2rad(-90)),
    dw.SE2Transform(end, np.deg2rad(-90)),
]

start = [-half_tile, y2]
start2 = [x1 - width / 2, y2]
end2 = [x1, y2 - width / 2]
middle2 = midpoints_curve(start2, end2)
end = [x1, -half_tile]
middle = midpoints_curve(start, end)
ctr_pts12 = [  # left to right
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(start2, np.deg2rad(0)),
    dw.SE2Transform(middle2, np.deg2rad(-45)),
    dw.SE2Transform(end2, np.deg2rad(-90)), 
    dw.SE2Transform(end, np.deg2rad(-90)),
]


test_lane7 = dw.LaneSegment(width, ctr_pts7)
test_lane8 = dw.LaneSegment(width, ctr_pts8)
test_lane9 = dw.LaneSegment(width, ctr_pts9)
test_lane10 = dw.LaneSegment(width, ctr_pts10)
test_lane11 = dw.LaneSegment(width, ctr_pts11)
test_lane12 = dw.LaneSegment(width, ctr_pts12)

# load the empty one-tile map again for better visualization
m = dw.construct_map(map_data)
m.set_object("Test Lane 7", test_lane7, ground_truth=gt)
m.set_object("Test Lane 8", test_lane8, ground_truth=gt)
m.set_object("Test Lane 9", test_lane9, ground_truth=gt)
m.set_object("Test Lane 10", test_lane10, ground_truth=gt)
m.set_object("Test Lane 11", test_lane11, ground_truth=gt)
m.set_object("Test Lane 12", test_lane12, ground_truth=gt)
ipython_draw_html(m);

INFO:duckietown_world: area: RectangularArea(pmin=[0. 0.],pmax=[1. 1.])
DEBUG:commons.fs:Written 0.1MB to: out/ipython_draw_html/140391898705728/drawing.html
INFO:duckietown_world:Written SVG
 fn_svg: out/ipython_draw_html/140391898705728/drawing.svg


### `Tiles.py`

The values computed for the control points can be found in the `tiles.py` file,
where the different one-lane and double-lane tiles are defined.

### Map loading

As soon as the tiles are defined, a map can be easily constructed in a yaml file (see folder `maps`).
To load the maps, use the functions defined in `map_loading`

### Parametrization, extraction and interpolation along lanes

Look into the jupyter notebook `playground-driving-game-mapsipynb`

# Double lane Roundabout

### Definition of lanes that stay parallel

In [4]:
# load a empty tile to display the lanes
map_data_yaml = """
tile_size: 1

tiles:
  - [asphalt]

"""
# load the map containing one tile using the duckietown world module
map_data = yaml.load(map_data_yaml, Loader=yaml.SafeLoader)
m = dw.construct_map(map_data)

# extract the tile size
tile_size=map_data["tile_size"]

width_old = 0.376  # width of lanes in duckietown world
width = width_old  # creating a tile where there are only one direction of lanes, take same with as in duckietown

# the hard coded values are all taken from the duckietown world module.
# I tried to scale the space between two lanes accordingly to the new lane width
width_between_lanes_old = 0.22 * 2 - width_old
width_between_lanes = width_between_lanes_old / 2

"""
Get the coordinates of the control points at the edge of a tile

                                x3  x4
                   ____________________ (0.5, 0.5)
                   |                  |
                   |        y         | y2
                   |        ^         |
                   |        |--> x    |
               y4  |                  | y1
               y3  |                  |
     (-0.5, -0.5)  --------------------
                       x1        x2
"""
y2 = (width_between_lanes / 2 + width / 2)
y1 = -y2
shift = 0.1
y3, y4 = np.array([y1, y2]) - shift

x1, x2, x3, x4 = y1, y2, -y4, -y3


half_tile = tile_size / 2


def midpoints_curve(start, end):
    """
    This functions calculates the coordinates of the control point in the the middle of a curve,
    which is perfectly circular
    """
    x1, y1 = start
    x2, y2 = end
    
    r = x2 - x1
    dr = (1-sqrt(2)/2) * r
    if y2 > y1:
        res = [x2 - dr, y1 + dr]
    else:
        res = [x2 - dr, y1 - dr]

    return [round(_, 3) for _ in res]



# roundabout lanes
start = [-half_tile, y3]
end = [x4, half_tile]
middle_right = midpoints_curve(start, end)

ctr_pts1 = [  # lane on the right
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(middle_right, np.deg2rad(45)),
    dw.SE2Transform(end, np.deg2rad(90)),
]

start = [-half_tile, y4]
end = [x3, half_tile]
middle_left = midpoints_curve(start, end)
ctr_pts2 = [  # lane on the left
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(middle_left, np.deg2rad(45)),
    dw.SE2Transform(end, np.deg2rad(90)),

]
    
#double go out
start = [-half_tile, y4]
end = [half_tile, y2]
middle = [0, (y2 + y4)/2]
ctr_pts3 = [  # lane on the right
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(middle, np.deg2rad(15)),
    dw.SE2Transform(end, np.deg2rad(0)),
]

start = [-half_tile, y3]
end = [half_tile, y1]
middle = [0, (y1 + y3)/2]
ctr_pts4 = [  # lane on the left
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(middle, np.deg2rad(15)),
    dw.SE2Transform(end, np.deg2rad(0)),
]


# double go in
start = [x1, -half_tile]
end = [x3, half_tile]
middle = [(x3 + x1)/2, 0]
ctr_pts5 = [  # lane on the right
    dw.SE2Transform(start, np.deg2rad(90)),
    dw.SE2Transform(middle, np.deg2rad(75)),
    dw.SE2Transform(end, np.deg2rad(90)),
]

start = [x2, -half_tile]
end = [x4, half_tile]
middle = [(x2 + x4)/2, 0]
ctr_pts6 = [  # lane on the left
    dw.SE2Transform(start, np.deg2rad(90)),
    dw.SE2Transform(middle, np.deg2rad(75)),
    dw.SE2Transform(end, np.deg2rad(90)),
]

#double go out lane change
start = [-half_tile, y4]
end = [half_tile, y1]
middle = [0, (y4 + y1)/ 2]
ctr_pts7 = [  # lane on the right
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(middle, np.deg2rad(-30)),
    dw.SE2Transform(end, np.deg2rad(0)),
]

start = [-half_tile, y3]
end = [half_tile, y2]
middle = [0, (y3 + y2) / 2]
ctr_pts8 = [  # lane on the left
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(middle, np.deg2rad(45)),
    dw.SE2Transform(end, np.deg2rad(0)),
]

# double go in lane change
start = [x1, -half_tile]
end = [x4, half_tile]
middle = [(x1 + x4) / 2, 0]
ctr_pts9 = [  # lane on the right
    dw.SE2Transform(start, np.deg2rad(90)),
    dw.SE2Transform(middle, np.deg2rad(45)),
    dw.SE2Transform(end, np.deg2rad(90)),
]

start = [x2, -half_tile]
end = [x3, half_tile]
middle = [(x2 + x3) / 2, 0]
ctr_pts10 = [  # lane on the left
    dw.SE2Transform(start, np.deg2rad(90)),
    dw.SE2Transform(middle, np.deg2rad(120)),
    dw.SE2Transform(end, np.deg2rad(90)),
]

# get the lane as a lane segment
test_lane1 = dw.LaneSegment(width, ctr_pts1)
test_lane2 = dw.LaneSegment(width, ctr_pts2)
test_lane3 = dw.LaneSegment(width, ctr_pts3)
test_lane4 = dw.LaneSegment(width, ctr_pts4)
test_lane5 = dw.LaneSegment(width, ctr_pts5)
test_lane6 = dw.LaneSegment(width, ctr_pts6)
test_lane7 = dw.LaneSegment(width, ctr_pts7)
test_lane8 = dw.LaneSegment(width, ctr_pts8)
test_lane9 = dw.LaneSegment(width, ctr_pts9)
test_lane10 = dw.LaneSegment(width, ctr_pts10)

# On a duckietown map the origin of the coordinate system is on the bottom left.
# In order to display the lanes which have been defined in the their original coordinate system
# (tile coordinate system) one has to move the lanes when their are placed on the empty one-tile map
gt = dw.SE2Transform([half_tile, half_tile], 0)
m.set_object("Test Lane 1", test_lane1, ground_truth=gt)
m.set_object("Test Lane 2", test_lane2, ground_truth=gt)
m.set_object("Test Lane 3", test_lane3, ground_truth=gt)
m.set_object("Test Lane 4", test_lane4, ground_truth=gt)
m.set_object("Test Lane 5", test_lane5, ground_truth=gt)
m.set_object("Test Lane 6", test_lane6, ground_truth=gt)
m.set_object("Test Lane 7", test_lane7, ground_truth=gt)
m.set_object("Test Lane 8", test_lane8, ground_truth=gt)
m.set_object("Test Lane 9", test_lane9, ground_truth=gt)
m.set_object("Test Lane 10", test_lane10, ground_truth=gt)
ipython_draw_html(m);

INFO:duckietown_world: area: RectangularArea(pmin=[0. 0.],pmax=[1. 1.])
DEBUG:commons.fs:Written 0.1MB to: out/ipython_draw_html/140391933987328/drawing.html
INFO:duckietown_world:Written SVG
 fn_svg: out/ipython_draw_html/140391933987328/drawing.svg
